In [1]:
import pickle

import numpy as np
import pandas as pd
import torch

import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
import torch.optim as optim

from sklearn.model_selection import train_test_split
from torch import nn
from torch_geometric.data import Data, Dataset, DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

In [2]:
def my_reshape(tensor):
    return torch.reshape(tensor, (tensor.shape[0], 1))

In [3]:
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер
def simple_preprocessing(batch):
    #spherical_radii = torch.Tensor(batch['spherical_domain_radii'])
    #spherical_radii = my_reshape(spherical_radii)
    
    tags = batch['tags'].long()
    tags = F.one_hot(tags, num_classes=3)
    
    atom_numbers = batch['atomic_numbers'].long()
    atom_numbers = F.one_hot(atom_numbers, num_classes=100)
    
    voronoi_volumes = torch.Tensor(batch['voloroi_volumes'])
    voronoi_volumes = my_reshape(voronoi_volumes)
    
    atom_features = (tags, atom_numbers, voronoi_volumes)#, spherical_radii)
    atom_embeds = torch.cat(atom_features, 1)
    
    edge_index = torch.Tensor(batch['edge_index_new']).long()
    
    distances = torch.Tensor(batch['distances_new'])
    distances = my_reshape(distances)
    
    angles = torch.Tensor(batch['contact_solid_angles'])
    angles = my_reshape(angles)
    
    edges_embeds = torch.cat((distances, angles), 1)
    
    
    return Data(x=atom_embeds, edge_index=edge_index, edge_attr=edges_embeds)

In [4]:
#датасет, который умеет возвращать эелемент и собственную длину
class Dataset(Dataset):

    def __init__(self, data, features_fields, target_field, type_='train', preprocessing=simple_preprocessing):
        
        self.data = data[features_fields]
        self.length = len(data)
        self.target = torch.Tensor(data[target_field].values)
        self.type_ = type_
        self.preprocessing = preprocessing

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        
        system = self.preprocessing(self.data.iloc[index])
        
        if self.type_ == 'train':
            y = self.target[index]
            
            return system, y

$$
\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right)
$$

Гамма лежит в апдейт, квадратик в aggr, а фи в месседж; в этом примере гамма и фи -- умножение на матрицу после конкатенации, а квадратик -- суммирование

In [46]:
class GConv(MessagePassing):
    def __init__(self, dim_atom=103, dim_edge=2, out_channels=2):
        super(GConv, self).__init__(aggr='add')  # "Add" aggregation
        self.phi_output = 3
        self.lin_phi = torch.nn.Linear(dim_atom*2+dim_edge, self.phi_output, bias=False)
        self.lin_gamma = torch.nn.Linear(dim_atom + self.phi_output, out_channels, bias=False)
        self.nonlin = nn.Sigmoid()

    def forward(self, batch):
        x = batch.x
        edge_index = batch.edge_index
        edge_attr = batch.edge_attr
        
        # x has shape [N -- количество атомов в системе(батче), in_channels -- размерность вектора-атома]
        # edge_index has shape [2, E] -- каждое ребро задаётся парой вершин

        # Start propagating messages. 
    
        return self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)  #не совсем понял что такое сайз

    def message(self, x, x_i, x_j, edge_attr):
        concatenated = torch.cat((x_i, x_j, edge_attr), 1)
        phi = self.lin_phi(concatenated)
        return self.nonlin(phi)
        
    def update(self, aggr_out, x, edge_attr, edge_index):
                
        concatenated = torch.cat((x, aggr_out), 1)

        return Data(x=self.nonlin(self.lin_gamma(concatenated)), edge_attr=edge_attr, edge_index=edge_index)

In [58]:
#собственно нейросеть
class ConvNN(nn.Module):
    
    def __init__(self, dim_atom=103, dim_edge=2):
        
        super().__init__()          
        self.conv_1 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_2 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_3 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_4 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_5 = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=dim_atom)
        self.conv_last = GConv(dim_atom=dim_atom, dim_edge=dim_edge, out_channels=2)
        
        self.lin = torch.nn.Linear(2, 1, bias=True)
        
    def forward(self, batch):
        convoluted_1 = self.conv_1(batch)
        convoluted_2 = self.conv_2(convoluted_1)
        convoluted_3 = self.conv_3(convoluted_2)
        convoluted_4 = self.conv_4(convoluted_3)
        convoluted_5 = self.conv_5(convoluted_4)
        convoluted_last = self.conv_last(convoluted_5).x
        scattered = scatter(convoluted_last, batch.batch, dim=0, reduce='sum')
        summed = scattered
        energy = self.lin(summed)
        
        return energy

In [59]:
#train -- ходим по батчам из итератора, обнуляем градиенты, предсказываем у, считаем лосс, считаем градиенты, делаем шаг оптимайзера, записываем лосс
def train(model, iterator, optimizer, criterion, print_every=10):
    
    epoch_loss = 0
    
    model.train()

    for i, (systems, ys) in enumerate(iterator):
        
        optimizer.zero_grad()
        predictions = model(systems).squeeze()
        loss = criterion(predictions.float(), ys.to(device).float())
        loss.backward()     
        
        optimizer.step()      
        
        epoch_loss += loss.item()  
        
        if not (i+1) % print_every:
            print(f'step {i} from {len(iterator)')
            print(f'Loss: {epoch_loss/i}')
        
    return epoch_loss / len(iterator)

In [60]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    
#    model.train(False)
    model.eval()  
    
    with torch.no_grad():
        for systems, ys in iterator:   

            predictions = model(systems).squeeze()
            loss = criterion(predictions.float(), ys.to(device).float())        

            epoch_loss += loss.item()  
            
    return epoch_loss / len(iterator)

In [61]:
def inferens(model, iterator):
    y = torch.tensor([])

#    model.train(False)
    model.eval()  
    
    with torch.no_grad():
        for systems, ys in iterator:   
          predictions = model(systemhs).squeeze()
          y = torch.cat((y, predictions))
      
    return y

## DATA

In [62]:
%%time
with open('/Users/humonen/Downloads/structures_train.pkl','rb') as f:
    data_ori = pickle.load(f)

CPU times: user 5.65 s, sys: 1.12 s, total: 6.77 s
Wall time: 6.9 s


In [63]:
%%time
#сливаем новые фичи и фичи из Data
for system in data_ori:
    for key in system['data']:
        system[key[0]] = key[1]
    del system['data']

CPU times: user 139 ms, sys: 4.01 ms, total: 143 ms
Wall time: 143 ms


In [64]:
%%time
df = pd.DataFrame(data_ori)

CPU times: user 69.5 ms, sys: 3.71 ms, total: 73.2 ms
Wall time: 71.9 ms


In [65]:
data_ori=[]

In [66]:
df

,id,covalent_radii,dipole_polarizability,electron_affinity,electronegativity,voloroi_volumes,voronoi_surface_areas,spherical_domain_radii,distances_new,contact_solid_angles,...,edge_index,fixed,force,natoms,pos,pos_relaxed,sid,tags,y_init,y_relaxed
0,0,"[1.21, 1.21, 1.21, 1.21, 1.21, 1.21, 1.21, 1.2...","[57.8, 57.8, 57.8, 57.8, 57.8, 57.8, 57.8, 57....","[0.43283, 0.43283, 0.43283, 0.43283, 0.43283, ...","[1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.6...","[89.3147203199472, 29.342061161368797, 15.7675...","[222.09641997716597, 75.76059098993872, 24.516...","[2.77296770148975, 1.913377530475639, 1.555579...","[4.704193115234375, 4.704193115234375, 2.84391...","[0.45406896877695196, 0.45406896877695196, 9.4...",...,"[[tensor(69), tensor(83), tensor(75), tensor(3...","[tensor(1.), tensor(0.), tensor(1.), tensor(0....","[[tensor(0.0767), tensor(0.0324), tensor(0.578...",86,"[[tensor(7.0256), tensor(0.), tensor(12.7346)]...","[[tensor(7.0256), tensor(0.), tensor(12.7346)]...",2472718,"[tensor(0), tensor(1), tensor(0), tensor(1), t...",6.282501,-0.025550
1,1,"[1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.7...","[112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 112...","[0.426, 0.426, 0.426, 0.426, 0.426, 0.426, 0.4...","[1.33, 1.33, 1.33, 1.33, 1.33, 1.33, 1.33, 1.3...","[13.858382002979626, 13.858382236327781, 95.59...","[35.868790697767324, 35.88634058092103, 377.46...","[1.4900744671796733, 1.4900744755429902, 2.836...","[3.6807498931884766, 3.6807498931884766, 3.680...","[0.006754217151977511, 0.006754217151977511, 0...",...,"[[tensor(55), tensor(63), tensor(52), tensor(4...","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[[tensor(-0.4060), tensor(-0.1663), tensor(-0....",85,"[[tensor(5.3127), tensor(11.3843), tensor(12.7...","[[tensor(5.3127), tensor(11.3843), tensor(12.7...",1747243,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",5.972082,-1.837069
2,2,"[1.45, 1.45, 1.45, 1.45, 1.45, 1.45, 1.45, 1.4...","[55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55.0, 55....","[1.3019999999999998, 1.3019999999999998, 1.301...","[1.93, 1.93, 1.93, 1.93, 1.93, 1.93, 1.93, 1.9...","[169.90046242317467, 16.909261919823788, 16.90...","[306.2641627020174, 34.77532226895127, 34.6920...","[3.435858874632096, 1.592252773326388, 1.59225...","[3.995115280151367, 3.995115280151367, 4.51798...","[7.348130132770407, 7.348130132770407, 3.46495...",...,"[[tensor(33), tensor(18), tensor(22), tensor(3...","[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[[tensor(0.0028), tensor(0.0790), tensor(0.005...",66,"[[tensor(1.9976), tensor(0.), tensor(12.5561)]...","[[tensor(1.9976), tensor(0.), tensor(12.5561)]...",1824821,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",0.686031,-0.152283
3,3,"[1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.75, 1.7...","[103.0, 103.0, 103.0, 103.0, 103.0, 103.0, 103...","[0.013999999999999999, 0.013999999999999999, 0...","[1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, 1.3, ...","[35.108055970210486, 22.470958553752013, 25.92...","[73.35361598401914, 33.916565450356856, 44.848...","[2.0312937125432513, 1.7505626701983967, 1.835...","[3.111886501312256, 3.111886501312256, 5.22778...","[15.71836643737836, 15.71836643737836, 0.27114...",...,"[[tensor(9), tensor(53), tensor(45), tensor(36...","[tensor(1.), tensor(1.), tensor(0.), tensor(1....","[[tensor(-0.2585), tensor(-0.0456), tensor(0.0...",62,"[[tensor(9.3824), tensor(12.5032), tensor(14.5...","[[tensor(9.3824), tensor(12.5032), tensor(14.5...",1324413,"[tensor(0), tensor(0), tensor(1), tensor(0), t...",2.222443,-6.227071
4,4,"[1.47, 1.47, 1.47, 1.47, 1.47, 1.47, 1.47, 1.4...","[79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79.0, 79....","[0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.55, 0.5...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...","[38.83699650086656, 14.355720997013236, 14.355...","[165.27906821064923, 34.537843861740484, 34.53...","[2.1008046766541075, 1.5076902742588287, 1.507...","[2.7617592811584473, 2.7617592811584473, 2.761...","[11.216767936903567, 11.216767936903567, 11.21...",...,"[[

In [67]:
df.columns

Index(['id', 'covalent_radii', 'dipole_polarizability', 'electron_affinity',
       'electronegativity', 'voloroi_volumes', 'voronoi_surface_areas',
       'spherical_domain_radii', 'distances_new', 'contact_solid_angles',
       'edge_index_new', 'atomic_numbers', 'cell', 'cell_offsets', 'distances',
       'edge_index', 'fixed', 'force', 'natoms', 'pos', 'pos_relaxed', 'sid',
       'tags', 'y_init', 'y_relaxed'],
      dtype='object')

In [68]:
#делим на обучующую и валидационную выборки
df_train, df_val = train_test_split(df, test_size=0.15)
df = []

In [69]:
#сбрасываем индексы
df_train = df_train.reset_index()
df_val = df_val.reset_index()

In [70]:
batch_size = 64
num_workers = 0

In [71]:
# features_cols = ['voloroi_volumes', 'voronoi_surface_areas', 'electronegativity', 
#                  'dipole_polarizability', 'edge_index_new', 'distances_new', 'contact_solid_angles']

features_cols = ['atomic_numbers', 'edge_index_new', 'distances_new', 
                 'contact_solid_angles', 'tags', 'voloroi_volumes', 'spherical_domain_radii']
target_col = 'y_relaxed'

In [72]:
#инициализируем тренировочный датасети и тренировочный итератор
training_set = Dataset(df_train, features_cols, target_col)
training_generator = DataLoader(training_set, batch_size=batch_size, num_workers=num_workers)

In [73]:
#инициализируем валидационный датасет и валидационный итератор
valid_set = Dataset(df_val, features_cols, target_col)
valid_generator = DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers)

In [74]:
df_train = []
df_val = []

## MODEL

In [75]:
#tensorboard writer
writer = SummaryWriter('/Users/humonen/Documents/our_base_model/tensorboard_logs')

In [76]:
#хочется уметь рисовать граф модели
if False:
    trace_system = []
    writer.add_graph(CGConv, trace_system)

In [77]:
writer.close()

In [78]:
#чтобы тензор по умолчанию заводился на куде
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print('cuda')

In [79]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cpu


In [80]:
#model
model = ConvNN(dim_atom=training_set[0][0].x.shape[1],dim_edge=training_set[0][0].edge_attr.shape[1])

#optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.L1Loss()

#переносим на куду если она есть
model = model.to(device)
criterion = criterion.to(device)

## Training

In [81]:
%%time
loss = []
loss_eval = []
epochs = 3
print(f'Start training model {str(model)}')
for i in range(epochs):
    print(f'epoch {i}')
    loss.append(train(model, training_generator, optimizer, criterion))
    loss_eval.append(evaluate(model, valid_generator, criterion))

Start training model ConvNN(
  (conv_1): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_2): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_3): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_4): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_5): GConv(
    (lin_phi): Linear(in_features=210, out_features=3, bias=False)
    (lin_gamma): Linear(in_features=107, out_features=104, bias=False)
    (nonlin): Sigmoid()
  )
  (conv_last): GConv(
    (lin_phi): Linear(in

In [82]:
loss_eval

[1.8168505529562633, 1.8069604287544887, 1.7993644177913666]